In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import time
import math
from math import radians
import gc
import seaborn as sns
from matplotlib import pyplot
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score
from IPython.core.display import HTML
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier, RandomTreesEmbedding
from lightgbm import LGBMClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#example everson griffen visualiziation
df1 = total_rolling_8.query('gameId == 2021101702 & playId == 2678 & nflId == 35539.0')

df1['modified_o'] = df1['o_deg_std']

df1.loc[df1.frameId > 21, 'modified_o'] = df1.modified_o + 360

In [ ]:
#side by side orientation plot
plt.rcParams.update(plt.rcParamsDefault)
fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(1, 2, 2, projection='polar')
c = ax.plot(df1['o_rad_std'], df1['frameId'],  '--o', alpha=0.75)
plt.title('Orientation')

ax = fig.add_subplot(1, 2, 1)
plt.plot(df1.frameId, df1.modified_o)
plt.title('Orientation')
plt.xlabel('Frame')

In [ ]:
#rolling orientation sum plot
plt.plot(df1.frameId, df1.rolling_o)
plt.title('Orientation Change Rolling Sum')
plt.scatter(17, 266.100006, color = 'red')
plt.xlabel('Frame')

In [ ]:
#feature importances
imp = {key:importance for key, importance in zip(X.columns, spin_model_1.feature_importances_)}

sorted_features = list(dict(sorted(imp.items(), key= lambda item:item[1], reverse = True)).items())

top_features_df = pd.DataFrame(top_features, columns = ['feature', 'importance'])

top_features_df.sort_values('importance', inplace = True)

ax = top_features_df.plot(kind='barh', x='feature', y='importance', figsize=(10, 8), 
                          title='Top Feature Importances')

In [ ]:
#display the top 10 spin move blockers
top_10_blockers = total_score_df.query('(num_spins_blocked >= 6) & (overall_total_blocks >= 100)').sort_values('avg_block_grade', ascending = False).drop_duplicates('nflId')[['displayName','PlayerTeam','num_spins_blocked','avg_block_grade']].head(10)
top_10_blockers = top_10_blockers.reset_index(drop = True).reset_index(drop = False)
top_10_blockers['index'] = top_10_blockers['index'] + 1

top_10_blockers['avg_block_grade'] = top_10_blockers.avg_block_grade.astype(float).round(2)

top_10_blockers = top_10_blockers.rename(
    columns = {'index': 'Rank', 'displayName': 'Name','num_spins_blocked': '# Spin Moves Blocked','avg_block_grade':'Average Block Grade','playerTeam': 'Team'})

In [ ]:
#top 10 spin move blockers table
fig, axs = plt.subplots(1, 1, figsize=(15, 5))
title_string = "Top 10 Spin Move Blockers"
subtitle_string = "Mimimum of 5 spins blocked, 100 total blocks"

axs.axis('off')

the_table = axs.table(cellText=top_10_blockers.values, colLabels=top_10_blockers.columns, loc='center', cellLoc='center')
the_table.auto_set_font_size(False)
the_table.scale(1, 2)
the_table.set_fontsize(14)

plt.suptitle(title_string, fontsize=18)
plt.title(subtitle_string, fontsize=10)

In [ ]:
#display the top 10 spin move rushers
top_10_rushers = total_score_df.query('(num_spins_rushed >= 6) & (blocked_overall_total_rushes >= 30)').sort_values('avg_rush_grade', ascending = False).drop_duplicates('blocked_nflId')[['blocked_displayName','defensiveTeam','num_spins_rushed','avg_rush_grade']].head(10)
top_10_rushers = top_10_rushers.reset_index(drop = True).reset_index(drop = False)
top_10_rushers['index'] = top_10_rushers['index'] + 1

top_10_rushers['avg_rush_grade'] = top_10_rushers.avg_rush_grade.astype(float).round(2)

top_10_rushers = top_10_rushers.rename(
    columns = {'index': 'Rank', 'blocked_displayName': 'Name','num_spins_rushed': '# Spin Moves Used','avg_rush_grade':'Average Rush Grade','defensiveTeam': 'Team'})

In [ ]:
#top 10 spin move rushers table
fig, axs = plt.subplots(1, 1, figsize=(15, 5))
title_string = "Top 10 Spin Move Rushers"
subtitle_string = "Mimimum of 6 spins rushed, 30 total rushes"


# axs.axis('tight')
axs.axis('off')

the_table = axs.table(cellText=top_10_rushers.values, colLabels=top_10_rushers.columns, loc='center', cellLoc='center')
the_table.auto_set_font_size(False)
the_table.scale(1, 2)
the_table.set_fontsize(14)

plt.suptitle(title_string, fontsize=18)
plt.title(subtitle_string, fontsize=10)

In [ ]:
#get qualified rushers
total_score_qualified_rushers = total_score_df.query('(num_spins_rushed >= 6) & (blocked_overall_total_rushes >= 30)').drop_duplicates('blocked_nflId').reset_index(drop = True)
qualified_rushers_names = total_score_qualified_rushers.loc[[0, 1, 2, 4, 5, 6, 7, 8, 10, 12, 15, 16, 17, 19, 22, 23,
                                                            24, 25, 26, 27, 28, 30, 31, 33, 34, 37, 41, 44, 46, 47,
                                                            ]]

In [ ]:
#create rusher spin score vs. prwr scatter plot
total_score_qualified_rushers.plot(kind='scatter', x='avg_rush_grade', y='blocked_overall_prwr',
           alpha=0.5, figsize=(18, 10))
plt.title('Mimimum of 6 spins rushed, 30 total rushes', fontsize = 15)
plt.suptitle('Overall Rush Win Rate vs. Average Spin Move Grade', fontsize = 20, weight = 'bold')

for i in range(qualified_rushers_names.shape[0]):
    plt.annotate(qualified_rushers_names.blocked_displayName.tolist()[i], (qualified_rushers_names.avg_rush_grade.tolist()[i], qualified_rushers_names.blocked_overall_prwr.tolist()[i]))

#add line of best fit to plot
# plt.plot(total_score_qualified_rushers['avg_rush_grade'], a*total_score_qualified_rushers['blocked_overall_prwr']+b)
plt.plot([total_score_qualified_rushers['avg_rush_grade'].mean(), total_score_qualified_rushers['avg_rush_grade'].mean()], [0, .3], 'k-', lw=1,dashes=[2, 2])
plt.plot([0,35], [total_score_qualified_rushers['blocked_overall_prwr'].mean(), total_score_qualified_rushers['blocked_overall_prwr'].mean()], 'k-', lw=1,dashes=[2, 2])

plt.text(20, 0.20, 'Good Spin Move Rusher, Good Overall Rusher',weight="bold", fontsize = 15)
plt.text(22, 0.05, 'Good Spin Move Rusher, Poor Overall Rusher',weight="bold", fontsize = 15)
plt.text(0, 0.05, 'Poor Spin Move Rusher, Poor Overall Rusher',weight="bold", fontsize = 15)
plt.text(0, 0.28, 'Poor Spin Move Rusher, Good Overall Rusher',weight="bold", fontsize = 15)

In [ ]:
#get qualified blockers
total_score_qualified_blockers = total_score_df.query('(num_spins_blocked >= 5) & (overall_total_blocks >= 100)').drop_duplicates('nflId').reset_index(drop = True)
qualified_blockers_names = total_score_qualified_blockers.loc[[2, 3, 4, 9, 10, 11, 14, 15, 16, 17, 18, 19, 21, 
                                                               25, 26, 27, 29, 30, 31, 34, 35, 36, 37, 41, 44,
                                                               45, 46, 47, 48, 49, 53, 54, 60,68, 70, 71, 73, 75, 76, 
                                                               79, 81]]

In [ ]:
#create blocker spin score vs. pbwr scatter plot

total_score_qualified_blockers.plot(kind='scatter', x='avg_block_grade', y='overall_pbwr',
           alpha=0.5, figsize=(18, 10))
  
plt.title('Mimimum of 5 spins blocked, 100 total blocks', fontsize = 15)
plt.suptitle('Overall Block Win Rate vs. Average Spin Move Block  Grade', fontsize = 20, weight = 'bold')
for i in range(qualified_blockers_names.shape[0]):
    plt.annotate(qualified_blockers_names.displayName.tolist()[i], (qualified_blockers_names.avg_block_grade.tolist()[i], qualified_blockers_names.overall_pbwr.tolist()[i]))
#add line of best fit to plot
# plt.plot(total_score_qualified_blockers['avg_block_grade'], a*total_score_qualified_blockers['overall_pbwr']+b) 
plt.text(90, 0.98, 'Good Spin Move Blocker, Good Overall Blocker',weight="bold", fontsize = 15)
plt.text(90, 0.83, 'Good Spin Move Blocker, Poor Overall Blocker',weight="bold", fontsize = 15)
plt.text(70, 0.83, 'Poor Spin Move Blocker, Poor Overall Blocker',weight="bold", fontsize = 15)
plt.text(70, 0.98, 'Poor Spin Move Blocker, Good Overall Blocker',weight="bold", fontsize = 15)
plt.plot([total_score_qualified_blockers['avg_block_grade'].mean(), total_score_qualified_blockers['avg_block_grade'].mean()], [.75, .975], 'k-', lw=1,dashes=[2, 2])
plt.plot([70,100], [total_score_qualified_blockers['overall_pbwr'].mean(), total_score_qualified_blockers['overall_pbwr'].mean()], 'k-', lw=1,dashes=[2, 2])